In [1]:
#import chrom drivers
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pygsheets
import time
import pandas as pd
from IPython.display import clear_output

# Craigslist Ad Scraper

### Function for Scraper Engine

In [2]:
# Parses html
def get_updated_html():
    thesoup = BeautifulSoup(browser.html, 'html.parser')

    return thesoup

In [3]:
# Scrape data from ad and populate the google sheet
def scrape_ad(row_count,the_html,current_link):
    #print(row_count)
    can_reply=True
    #get html and pull data from the page
    #the_html=get_updated_html()


    #job type
    job_title=""
    the_cats=the_html.find_all("span")
    for cat in the_cats:
        if str(cat).__contains__("job title"):
            job_title=str(cat).split('<b>')[1].split('</b>')[0]
            #print(job_title)

    #compensation
    compensation=""
    the_cats=the_html.find_all("span")
    for cat in the_cats:
        if str(cat).__contains__("compensation"):
            compensation=(str(cat).split('<b>')[1].split('</b>')[0])
            #print(compensation)

    #employment type
    employment_type=""
    the_cats=the_html.find_all("span")
    for cat in the_cats:
        if str(cat).__contains__("employment type"):
            employment_type=(str(cat).split('<b>')[1].split('</b>')[0])
            #print(employment_type)

    #company name
    if(len(the_html.find_all('h2'))!=0):
        company_name=str(the_html.find_all('h2')[0]).split('>')[1].split('<')[0]
    else:
        company_name=input('Enter company name manually (or leave empty to skip): ')
    if company_name=='':
        can_reply=False
        company_name='skip'

    #title
    title=str(the_html.find_all('span',id="titletextonly")[0]).split('>')[1].split('<')[0]
    #print(title)

    #posting body
    posting_body=str(the_html.find_all('section',id="postingbody")).split('</div>\n')[2].split('</section')[0]
    #print(posting_body)

    #date posted
    posted=""
    the_cats=the_html.find_all("p")

    for cat in the_cats:
        if str(cat).__contains__('date timeago"') & str(cat).__contains__('Posted'):
            posted=(str(cat).split('datetime="')[1].split('"')[0])
            #print(posted)

    #post id
    post_id=""
    the_cats=the_html.find_all("p")
    #print(the_cats)
    for cat in the_cats:
        if str(cat).__contains__('post id'):
            post_id=(str(cat).split('post id: ')[1].split('</p>')[0])
            #print(post_id)
    
    #city location and area
    city=the_html.find_all('a')[1].text
    city_area=the_html.find_all('a')[2].text        
            
    #post email
    contbutt=""
    if company_name!='skip' and (post_id not in stored_ids):
        if company_name+city+city_area not in stored_restaurants and str(job_title).__contains__('washer')==False \
        and str(job_title).__contains__('cleaner')==False and str(job_title).__contains__('Cleaner')==False:
            if len(browser.find_by_tag('button'))!=0:
                browser.find_by_tag('button')[0].click()
                time.sleep(.5)
                #temp_button=browser.find_by_text('show address')
                temp_button=browser.find_by_text('email')
                if (len(temp_button)==0):
                    temp_button=browser.find_by_text('show address')
                    if (len(temp_button)==0):
                        contbutt=input('<Return> to Continue when Captcha Code is entered (or <x> to skip ad)') #if needed to verify  
                
                if contbutt=="":
                    temp_button=browser.find_by_text('email')
                    if (len(temp_button)!=0):
                        browser.find_by_text('email').click()
                    
                    temp_button=browser.find_by_text('show address')
                    if (len(temp_button)!=0):
                        browser.find_by_text('show address').click()     
                    
                    time.sleep(.5)
                    the_html=get_updated_html()
                    email_link=""
                    the_links=the_html.find_all('a')

                    for link in the_links:
                        if str(link).__contains__('mailto'):
                            email_link=(str(link).split('mailto:')[1].split('?')[0])

                    browser.find_by_tag('button')[0].click() 
                else:
                    can_reply=False
                    email_link=""
                
            else:
                email_link=input('Manually enter email or leave blank to skip')
        else:
            can_reply=False
            email_link=""
    else:
        email_link=""      
    
    if email_link!="" and can_reply==True and (post_id not in stored_ids):
        print('success')
        wks.update_value(f'A{row_count}', company_name)
        wks.update_value(f'B{row_count}', job_title)
        wks.update_value(f'C{row_count}', compensation)
        wks.update_value(f'D{row_count}', employment_type)
        wks.update_value(f'E{row_count}', title)
        wks.update_value(f'F{row_count}', posted)
        wks.update_value(f'G{row_count}', post_id)

        wks.update_value(f'H{row_count}', email_link)
        wks.update_value(f'I{row_count}', city)
        wks.update_value(f'J{row_count}', city_area)
        wks.update_value(f'K{row_count}', "No")
        wks.update_value(f'L{row_count}', "None")
        wks.update_value(f'O{row_count}', current_link)
        
        stored_restaurants.append(company_name+city+city_area)
        return True
    else: 
        print('unsuccessful') #if it wasn't able to scrape ad it will return this
        return False
        time.sleep(.2)
        

In [4]:
# Function for scraper to push the 'next' button
def click_next():
    next_buttons=browser.find_by_tag('button')
    for button in next_buttons:
        if str(button['class']).__contains__('bd-button cl-next-page'):
            temp_button=button
            
    temp_button.click()

In [5]:
# Before scraping, thiscd hecks the csv list of already scraped ads to ensure it has not been visited 
def get_sheet_values_for_comparison():
    a=wks.get_col(1)
    b=wks.get_col(9)
    c=wks.get_col(10)

    d=[]
    for count, aa in enumerate(a):
        if aa !="":
            d.append(a[count]+b[count]+c[count])
            
    return d

In [6]:
# Pulls ad ids for further comparison to ensure no ad is scraped twice
def get_stored_ids():
    a=wks.get_col(7)

    d=[]
    for count, aa in enumerate(a):
        if aa !="":
            d.append(a[count])
        
    return d

In [7]:
# Drives the webscraper
def scrap_cragsl(scroll_pages_deep,avoids,the_city):
    input('Make sure you are on the top food/beverage/hospitality page (press return when you are ready)')
    
    row_count=len(wks.get_values_batch('A')[0])+1 #finds first empty row
    df=pd.read_csv(f'visted_ads_{the_city}.csv')

    link_list=[] #stores links to search


    #scroll through pages and grab links
    for x in range(scroll_pages_deep):
        #grab ad links           
        page_links=browser.find_by_tag('a')                   
        for link in page_links:
            if(link.has_class("titlestring")==True):
                in_avoid_list=False
                for avoid in avoids:
                    if (str(link['href']).upper().__contains__(avoid.upper())==True):
                        in_avoid_list=True              
                if (link['href'] not in df['links'].tolist() and in_avoid_list==False):
                    link_list.append(link['href'])

    limit_search_to_ads=input(f'Found {len(link_list)} new ads. How many do you want to scrape? ')

    #scan ads
    for x in range(len(link_list)):
        print(x)
        if x==int(limit_search_to_ads):
            break

        #visit ad    
        if(link_list[x] not in df['links'].tolist()):
            skip_ad=input(f'Scrape: {link_list[x]}? (leave BLANK to scrape ad)')
            if(skip_ad==''):
                the_url=link_list[x]
                browser.visit(the_url)
                time.sleep(1)
                temp_html=get_updated_html()
                check_if_added=scrape_ad(row_count,temp_html,link_list[x])
                #append df
                df.loc[len(df.index)]=link_list[x]
                time.sleep(1)
            else:
                print('skipped ad')
                time.sleep(.2)
                check_if_added=False
                df.loc[len(df.index)]=link_list[x]
                #write back to csv to avoid visiting pages again
            df.to_csv(f'visted_ads_{the_city}.csv',index=False)
            if check_if_added==True:
                row_count=row_count+1
            clear_output(wait=False)


### Initiate Chrome Driver and Run Scraper

In [25]:
# Enter the city to scrape 
city_csv='miami'


#Sets up engine
executable_path= {'executable_path': ChromeDriverManager().install()}
browser= Browser('chrome', **executable_path, headless=False)
time.sleep(3)
browser.visit(f'https://{city_csv}.craigslist.org/search/fbh')


In [26]:
# Connect to Google Sheets
client = pygsheets.authorize(service_file='cred4.json')
sh=client.open('craigslist_data')
wks = sh[0]

In [4]:
# How many pages deep do you want to scroll back (usually 1 page is enough (about 100 ads))
pages_deep=1

#a list of job titles to avoid
titles_to_avoid=['doordash','instacart','domino','fishing','shopper','travel','dishwasher','McMenamins','smyrna',\
                'superica','manager']


#creates a list of already stored restaurants
stored_restaurants=get_sheet_values_for_comparison()
stored_ids=get_stored_ids()


scrap_cragsl(pages_deep,titles_to_avoid,city_csv)


46 ads scraped
Google Sheet updated


# Send Emails

This section pulls the stored emails from the Google Sheet, composes a unique cover letter using data from the ad, then attaches the appropriate resume for the position offered, before sending the email.

### email functions

In [28]:
import os
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

In [29]:
# You need to set up app password. See link below if you do not have an app password:
# https://help.warmupinbox.com/en/articles/4934806-configure-for-google-workplace-with-two-factor-authentication-2fa

def send_email(resume_ver,subject,ap_name,num,sender_email,new_title,sender_password):

    
    recipient_email = wks.get_value(f'H{num}')
    #recipient_email = 'julia.claira@berkeley.edu'
    subject = f"Re: Craigslist {wks.get_value(f'E{num}')}"
    body = compose_email(ap_name,num,new_title)


    with open(pull_resume(resume_ver,wks.get_value(f'I{num}')), "rb") as attachment:
        # Add the attachment to the message
        part = MIMEBase("application", "octet-stream")
        part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {os.path.basename(pull_resume(resume_ver,wks.get_value(f'I{num}')))}",
    )

    message = MIMEMultipart()
    message['Subject'] = subject
    message['From'] = sender_email
    message['To'] = recipient_email
    html_part = MIMEText(body)
    message.attach(html_part)
    message.attach(part)

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipient_email, message.as_string())
    server.quit()


In [30]:
# Generates the body of the email.
def compose_email(applicant,num,new_title):
    
    the_position=new_title
    the_company=wks.get_value(f'A{num}')


    if the_company!="":
        emailbody= f'\
Hi,\n\n\
I am interested in the {the_position} position that you posted on Craigslist. \
I have always been passionate about food and customer service, and I believe that I would make a great addition to your team at {the_company}. \
My resume is attached. \n \n\
Thank you, \n \n\
{applicant}'
        
    else:
        emailbody= f'\
Hi,\n\n\
I am interested in the {the_position} position that you posted on Craigslist. \
I have always been passionate about food and customer service, and I believe that I would make a great addition to your team. \
My resume is attached. \n \n\
Thank you, \n \n\
{applicant}'


    return emailbody


In [31]:
# Pulls the resume to attach based on city name and job position

def pull_resume(x,city):
    match x:
        case 1:
            return f"241_resumes/{city}/server/Charles_Miller_Resume.pdf"
        case 2:
            return f"241_resumes/{city}/server/Claire_Miller_Resume.pdf"
        case 3:
            return f"241_resumes/{city}/cook/Charles_Miller_Resume.pdf"
        case 4:
            return f"241_resumes/{city}/cook/Claire_Miller_Resume.pdf"
        case 5:
            return f"241_resumes/{city}/server/Cypress_Miller_Resume.pdf"
        case 6:
            return f"241_resumes/{city}/cook/Cypress_Miller_Resume.pdf"


### sends the emails

In [32]:
# Connect to Google Sheets
client = pygsheets.authorize(service_file='cred4.json')
sh=client.open('craigslist_data')
wks = sh[0]

In [37]:
# Send emails between these rows 
range_begin=691
range_end=733

In [3]:
# After Sending emails this will update the Google Sheet

#Import passwords for emails
from craigslist_p import psw

#pull columns
email_col=wks.get_col(8)
resume_sent_col=wks.get_col(11)
title_col=wks.get_col(5)
length_of_sh=len(wks.get_values_batch('A')[0])+1

email_version=1
version=1 #1 for Charles and 2 for Claire 3 for Cypress
applicant_name=["","Charles","Claire","Charles","Claire","Cypress","Cypress"]

city_opt1 ={"new york":["","c.miller.37949","c.miller.51355","c.miller.62235"],\
               "south florida":["","c.miller.25804","c.miller.78882","c.miller.81313"],\
               "chicago":["","c.miller.14523","c.miller.40104","c.miller.14523"]}

city_opt2={"new york":["","c.miller.37949","c.miller.51355","c.miller.62235"],\
                       "south florida":["","c.miller.25804","c.miller.78882","c.miller.81313"],\
                       "chicago":["","c.miller.40104","c.miller.14523","c.miller.40104"]}

sender_email_d = city_opt1

the_pswrds={'c.miller.37949@gmail.com':psw[1],"c.miller.51355@gmail.com":psw[2],\
                "c.miller.25804@gmail.com":psw[3],\
                 "c.miller.78882@gmail.com":psw[4],'c.miller.14523@gmail.com':psw[5],\
            "c.miller.40104@gmail.com":psw[6],"c.miller.62235@gmail.com":psw[7],\
           "c.miller.81313@gmail.com":psw[8],"c.miller.99811@gmail.com":psw[9]} 

#send emails
found_cities=True
city_count=1
city_e='new york'
city_pop=['new york','south florida','chicago']
sf=1

while found_cities==True:
    
    found_cities=False
    for i in range(range_begin,range_end):

        if city_count <=3:
            city_e='new york'                  
        elif city_count<=6 and city_count>3:
            city_e='south florida'                       
        else:            
            city_e='chicago'           
        
        if(resume_sent_col[i-1]=='No' and wks.get_value(f'I{i}')==city_e):
            found_cities=True

            #update resume sent column
            wks.update_value(f'K{i}', 'Yes')
            #add time resume sent
            wks.update_value(f'M{i}', str(datetime.now()))

            recorded_title=wks.get_value(f'B{i}')

            the_title=input(f'Title: {recorded_title} - Leave BLANK to keep as is, or type new title: ')
            which_resume=input('Server or Cook resume (enter "s" or "c")')
            if (which_resume !='s'):
                which_resume='c'
                if version!=5:
                    version=version+2
                else:
                    version=version+1

            #add resume version sent
            wks.update_value(f'N{i}', version)

            if the_title=='':
                    the_title=wks.get_value(f'B{i}')

            sender_version=sender_email_d[wks.get_value(f'I{i}')][email_version]+'@gmail.com'

            #send email
            send_email(version,title_col[i],applicant_name[version],i,sender_version,the_title,\
                      the_pswrds[sender_version])
            time.sleep(.2)

            wks.update_value(f'P{i}', sender_version)
            wks.update_value(f'Q{i}', which_resume)

            print(city_e)
            print(applicant_name[version])
            print(f'resume version: {version}')
            print(f'row {i}')
            time.sleep(3)

            #update email version
            if email_version !=3:
                email_version=email_version+1
            else:
                email_version =1

            if version==2 or version==4:
                    version=5
            elif version==1 or version==3:
                    version=2
            else:
                version=1

            clear_output(wait=False)
            city_count=city_count+1
            if city_count>=10:
                city_count=1
                if sf==1:
                    sender_email_d=city_opt2
                    sf=2
                    
                else:
                    sf=1
                    sender_email_d=city_opt1
                    
    if(city_e not in city_pop and found_cities==False):
        city_count=city_count+3
                
    elif (found_cities==False and city_e in city_pop):
        city_pop.remove(city_e)
        print('city_pop')
        city_count=city_count+3
        
    if len(city_pop)==0:
        found_cities=False
    else:
        found_cities=True
    if city_count>=10:
        city_count=1
        if sf==1:
            sender_email_d=city_opt2
            sf=2
            print('shift sf')
        else:
            sf=1
            sender_email_d=city_opt1
    
    print(city_count)
    print(f'sf: {sf}')
    time.sleep(1)
    resume_sent_col=wks.get_col(11)
    time.sleep(1)
    


34 emails sent!
Google Sheet updated
